In [3]:
import sys
import glob
sys.path.append("../../")

from python.src.train import experiment_tracking_train
from python.src.sat_representations import VCG, LCG, SATRepresentation
from pathlib import Path
NUM_EPOCHS = 5  # 10
f = 0.0000001
alpha = 1
beta = 0
gamma = 0
batch_size = 2
# path = "/Users/p403830/Library/CloudStorage/OneDrive-PorscheDigitalGmbH/GIT_SAT_ML/data/blocksworld"
path = "/Users/p403830/Library/CloudStorage/OneDrive-PorscheDigitalGmbH/programming/generateSAT/random_3SAT_subset"
# path = "/Users/p403830/Library/CloudStorage/OneDrive-PorscheDigitalGmbH/GIT_SAT_ML/data/LLL_sample_one"
# path = "/Users/p403830/Library/CloudStorage/OneDrive-PorscheDigitalGmbH/programming/generateSAT/samples_LLL_n80/"
N_STEPS_MOSER = 100
N_RUNS_MOSER = 5
SEED = 0
graph_representation = "LCG"
network_type = "interaction"
# network_definition = get_network_definition(network_type = network_type, graph_representation = graph_representation) #network_definition_interaction_new

MODEL_REGISTRY = Path("../../../mlrun_save")
EXPERIMENT_NAME = "trash"

model_path = False # "../../params_save/" + EXPERIMENT_NAME
img_path = False # model_path + "_plot"

match graph_representation:
    case "LCG":
            rep = LCG
    case "VCG":
            rep = VCG
print(glob.glob(glob.escape(path) + "/*.cnf"))


experiment_tracking_train(
        MODEL_REGISTRY,
        EXPERIMENT_NAME,
        batch_size,
        f,
        alpha,
        beta,
        gamma,
        NUM_EPOCHS,
        N_STEPS_MOSER,
        N_RUNS_MOSER,
        path,
        img_path="show",
        model_path=model_path,
        graph_representation=rep,
        network_type=network_type,
    )

['/Users/p403830/Library/CloudStorage/OneDrive-PorscheDigitalGmbH/programming/generateSAT/random_3SAT_subset/random_KCNF3_100_119_9283351.cnf', '/Users/p403830/Library/CloudStorage/OneDrive-PorscheDigitalGmbH/programming/generateSAT/random_3SAT_subset/random_KCNF3_100_119_6639307.cnf', '/Users/p403830/Library/CloudStorage/OneDrive-PorscheDigitalGmbH/programming/generateSAT/random_3SAT_subset/random_KCNF3_100_119_4391198.cnf', '/Users/p403830/Library/CloudStorage/OneDrive-PorscheDigitalGmbH/programming/generateSAT/random_3SAT_subset/random_KCNF3_100_100_3602064.cnf', '/Users/p403830/Library/CloudStorage/OneDrive-PorscheDigitalGmbH/programming/generateSAT/random_3SAT_subset/random_KCNF3_100_100_2658295.cnf', '/Users/p403830/Library/CloudStorage/OneDrive-PorscheDigitalGmbH/programming/generateSAT/random_3SAT_subset/random_KCNF3_100_100_2924554.cnf']
pad_nodes, pad_edges 320 457
n_node, n_edge 300 400


AttributeError: 'list' object has no attribute 'values'

In [23]:
import jax
import jax.numpy as jnp
import jraph
from jraph import GraphsTuple
from jraph import get_graph_padding_mask

key = jax.random.PRNGKey(0)
graph_1 = GraphsTuple(nodes=jax.random.normal(key, (3, 64)),
                  edges=jax.random.normal(key, (5, 64)),
                  senders=jnp.array([0,0,1,1,2]),
                  receivers=jnp.array([1,2,0,2,1]),
                  n_node=jnp.array([3]),
                  n_edge=jnp.array([6]),
                  globals=jnp.array([0,0,0,1,2,0,2]))
graph_2 = GraphsTuple(nodes=jax.random.normal(key, (5, 64)),
                  edges=jax.random.normal(key, (10, 64)),
                  senders=jnp.array([0,0,1,1,2,2,3,3,4,4]),
                  receivers=jnp.array([1,2,0,2,1,0,2,1,3,2]),
                  n_node=jnp.array([5]),
                  n_edge=jnp.array([13]),
                  globals=jnp.array([0,3,0,1,2,0,3]))
batch = jraph.batch([graph_1, graph_2])

In [ ]:
print(graph_1.senders)
print(graph_2.senders)

print(batch.senders)

0
[0 0 1 1 2 2 3 3 4 4]
[0 0 1 1 2 3 3 4 4 5 5 6 6 7 7]


In [10]:
import sys
import glob
sys.path.append("../../")
from pysat.formula import CNF
import numpy as np
from python.src.sat_instances import get_problem_from_cnf
from python.src.sat_representations import VCG, LCG, SATRepresentation
def do():
        n = 10
        m = 25
        k = 2
        mask = LCG.get_mask(n, 2*n + m)
        H = CNF()
        counter = 0
        for i in range(m):
            c = []
            for j in range(k):
                c.append(-1*((counter + j)%n + 1))
            counter += 1
            H.append(c)
        problem = get_problem_from_cnf(H, LCG, include_constraint_graph=0)
        g = problem.graph
        decoded_nodes = []
        for i in range(2*n + m):
            if i%2 == 0 and i< 2*n:
                decoded_nodes.append(10000)
            else:
                decoded_nodes.append(0)
        decoded_nodes = np.array([decoded_nodes]).T
        print(decoded_nodes)
        assert decoded_nodes.shape == (2*n + m, 1)
        mask = LCG.get_mask(n, 2*n + m)
        neighbors_list = LCG.get_constraint_graph(n, m, g.senders, g.receivers)
        print(neighbors_list)
        loss = LCG.local_lovasz_loss(decoded_nodes, mask, g, neighbors_list)
        print(loss)

In [11]:
do()

[[10000]
 [    0]
 [10000]
 [    0]
 [10000]
 [    0]
 [10000]
 [    0]
 [10000]
 [    0]
 [10000]
 [    0]
 [10000]
 [    0]
 [10000]
 [    0]
 [10000]
 [    0]
 [10000]
 [    0]
 [    0]
 [    0]
 [    0]
 [    0]
 [    0]
 [    0]
 [    0]
 [    0]
 [    0]
 [    0]
 [    0]
 [    0]
 [    0]
 [    0]
 [    0]
 [    0]
 [    0]
 [    0]
 [    0]
 [    0]
 [    0]
 [    0]
 [    0]
 [    0]
 [    0]]
[[20 20 20 20 20 20 20 21 21 21 21 21 21 21 21 22 22 22 22 22 22 22 22 23
  23 23 23 23 23 23 23 24 24 24 24 24 24 24 25 25 25 25 25 25 26 26 26 26
  26 27 27 27 27 27 28 28 28 28 28 29 29 29 29 29 29 30 30 30 30 30 30 30
  31 31 31 31 31 31 31 31 32 32 32 32 32 32 32 32 33 33 33 33 33 33 33 33
  34 34 34 34 34 34 34 35 35 35 35 35 35 36 36 36 36 36 37 37 37 37 37 38
  38 38 38 38 39 39 39 39 39 39 40 40 40 40 40 40 40 41 41 41 41 41 41 41
  41 42 42 42 42 42 42 42 42 43 43 43 43 43 43 43 43 44 44 44 44 44 44 44]
 [41 31 21 40 39 30 29 42 32 22 41 40 31 30 20 43 33 23 42 41 32 31 21 44
 

TypeError: logistic does not accept dtype int32. Accepted dtypes are subtypes of floating, complexfloating.